In [1]:
import torch
from fairseq.models.bart import BARTModel
import shutil
import os

In [8]:
class args:
#     model_dir = '/home/lptang/fairseq/checkpoints/denoising_bart/shuffle_bleu_iclr/ng_shuffle8_ba_0.0_bleu12_8_lr_1e-06/'
    model_file = 'checkpoint_best.pt'
    model_dir = '/home/lptang/fairseq/checkpoints/denoising_bart/shuffle/cr_shuffle8_ba_1.0_cetf_8/'

In [9]:
if 'dict.de.txt' not in os.listdir(args.model_dir):
    shutil.copyfile('/home/lptang/fairseq/multi30k-bin/shuffle3/dict.en.txt',args.model_dir+'dict.en.txt')
    shutil.copyfile('/home/lptang/fairseq/multi30k-bin/shuffle3/dict.de.txt',args.model_dir+'dict.de.txt')
bart = BARTModel.from_pretrained(
    args.model_dir,
    checkpoint_file=args.model_file,
    data_name_or_path=args.model_dir,
)

In [10]:
bart = bart.eval()
bart = bart.cuda().half()

In [11]:
infile='/home/lptang/fairseq/examples/translation/multi30k_shuffle3/test.de'
outfile="bart_hypo_2.txt"
bsz=32
n_obs=None
count = 1


In [12]:
eval_kwargs = dict(beam=4, lenpen=2.0, max_len_b=30, min_len=2, no_repeat_ngram_size=3)

In [13]:
    with open(infile) as source, open(outfile, "w") as fout:
        sline = source.readline().strip()
        slines = [sline]
        for sline in source:
            if n_obs is not None and count > n_obs:
                break
            if count % bsz == 0:
                hypotheses_batch = bart.sample(slines, **eval_kwargs)
                for hypothesis in hypotheses_batch:
                    fout.write(hypothesis + "\n")
                    fout.flush()
                slines = []

            slines.append(sline.strip())
            count += 1

        if slines != []:
            hypotheses_batch = bart.sample(slines, **eval_kwargs)
            for hypothesis in hypotheses_batch:
                fout.write(hypothesis + "\n")
                fout.flush()